# SimPy: Treatment Centre

`simpy` uses process based model worldview.  Given its simplicity it is a highly flexible discrete-event simulation package.

Here we will take a look at code that implements a full `simpy` model including, time dependent arrivals, results collection, control of random numbers and multiple replications.

> Detailed documentation for `simpy` can be found here: https://simpy.readthedocs.io/en/latest/

---

## Imports

It is recommended that you use the provided conda virtual environment `os-sim`.  

>If you are running this code in **Google Colab** then `simpy` can be pip installed.

In [1]:
# if running in Google Colab.
#!pip install simpy==4.0.1

In [2]:
import simpy
simpy.__version__

'4.0.1'

In [3]:
import numpy as np
import pandas as pd
import itertools
import math
import matplotlib.pyplot as plt

In [4]:
# need numpy > v1.18
np.__version__

'1.19.2'

## Applied Example: A health clinic in the US.

This example is based on Nelson (2010) ex. 13 page 170.

## Constants

In [5]:
TRACE = True

# 4.8 minutes between arrivals (225 per 18 hours)
DEFAULT_MEAN_IAT = 4.8

DEFAULT_TRIAGE_MEAN = 3.0

DEFAULT_REG_MEAN = 5.0
DEFAULT_REG_VAR= 2.0

DEFAULT_EXAM_MEAN = 16.0
DEFAULT_EXAM_VAR = 3.0

DEFAULT_TRAUMA_MEAN = 90.0

DEFAULT_TRAUMA_TREAT_MEAN = 30.0
DEFAULT_TRAUMA_TREAT_VAR = 4.0

DEFAULT_NON_TRAUMA_TREAT_MEAN = 13.3
DEFAULT_NON_TRAUMA_TREAT_VAR = 2.0

DEFAULT_NON_TRAUMA_TREAT_P = 0.40

DEFAULT_PROB_TRAUMA = 0.30

# Resource counts
DEFAULT_N_TRIAGE = 10
DEFAULT_N_REG = 10
DEFAULT_N_EXAM = 20
DEFAULT_N_TRAUMA = 20
DEFAULT_N_CUBICLES_1 = 50
DEFAULT_N_CUBICLES_2 = 50

# default random number SET
DEFAULT_RNG_SET = None
N_STREAMS = 10

# default results collection period
DEFAULT_RESULTS_COLLECTION_PERIOD = 60 * 18

# number of replications.
DEFAULT_N_REPS = 5

## Utility functions

In [6]:
def trace(msg):
    '''
    Utility function for printing a trace as the
    simulation model executes.
    Set the TRACE constant to False, to turn tracing off.
    
    Params:
    -------
    msg: str
        string to print to screen.
    '''
    if TRACE:
        print(msg)

## Distribution classes

To help with controlling sampling `numpy` distributions are packaged up into classes that allow easy control of random numbers.

In [7]:
class Exponential:
    '''
    Convenience class for the exponential distribution.
    packages up distribution parameters, seed and random generator.
    '''
    def __init__(self, mean, random_seed=None):
        '''
        Constructor
        
        Params:
        ------
        mean: float
            The mean of the exponential distribution
        
        random_seed: int, optional (default=None)
            A random seed to reproduce samples.  If set to none then a unique
            sample is created.
        '''
        self.rng = np.random.default_rng(seed=random_seed)
        self.mean = mean
        
    def sample(self, size=None):
        '''
        Generate a sample from the exponential distribution
        
        Params:
        -------
        size: int, optional (default=None)
            the number of samples to return.  If size=None then a single
            sample is returned.
        '''
        return self.rng.exponential(self.mean, size=size)

    
class Bernoulli:
    '''
    Convenience class for the Bernoulli distribution.
    packages up distribution parameters, seed and random generator.
    '''
    def __init__(self, p, random_seed=None):
        '''
        Constructor
        
        Params:
        ------
        p: float
            probability of drawing a 1
        
        random_seed: int, optional (default=None)
            A random seed to reproduce samples.  If set to none then a unique
            sample is created.
        '''
        self.rng = np.random.default_rng(seed=random_seed)
        self.p = p
        
    def sample(self, size=None):
        '''
        Generate a sample from the exponential distribution
        
        Params:
        -------
        size: int, optional (default=None)
            the number of samples to return.  If size=None then a single
            sample is returned.
        '''
        return self.rng.binomial(n=1, p=self.p, size=size)

class Lognormal:
    """
    Encapsulates a lognormal distirbution
    """
    def __init__(self, mean, stdev, random_seed=None):
        """
        Params:
        -------
        mean: float
            mean of the lognormal distribution
            
        stdev: float
            standard dev of the lognormal distribution
            
        random_seed: int, optional (default=None)
            Random seed to control sampling
        """
        self.rng = np.random.default_rng(seed=random_seed)
        mu, sigma = self.normal_moments_from_lognormal(mean, stdev**2)
        self.mu = mu
        self.sigma = sigma
        
    def normal_moments_from_lognormal(self, m, v):
        '''
        Returns mu and sigma of normal distribution
        underlying a lognormal with mean m and variance v
        source: https://blogs.sas.com/content/iml/2014/06/04/simulate-lognormal
        -data-with-specified-mean-and-variance.html

        Params:
        -------
        m: float
            mean of lognormal distribution
        v: float
            variance of lognormal distribution
                
        Returns:
        -------
        (float, float)
        '''
        phi = math.sqrt(v + m**2)
        mu = math.log(m**2/phi)
        sigma = math.sqrt(math.log(phi**2/m**2))
        return mu, sigma
        
    def sample(self):
        """
        Sample from the normal distribution
        """
        return self.rng.lognormal(self.mu, self.sigma)

In [8]:
class Normal:
    '''
    Convenience class for the normal distribution.
    packages up distribution parameters, seed and random generator.
    '''
    def __init__(self, mean, sigma, random_seed=None):
        '''
        Constructor
        
        Params:
        ------
        mean: float
            The mean of the normal distribution
            
        sigma: float
            The stdev of the normal distribution
        
        random_seed: int, optional (default=None)
            A random seed to reproduce samples.  If set to none then a unique
            sample is created.
        '''
        self.rng = np.random.default_rng(seed=random_seed)
        self.mean = mean
        self.sigma = sigma
        
    def sample(self, size=None):
        '''
        Generate a sample from the normal distribution
        
        Params:
        -------
        size: int, optional (default=None)
            the number of samples to return.  If size=None then a single
            sample is returned.
        '''
        return self.rng.normal(self.mean, self.sigma, size=size)



## Model parameterisation

For convienience a container class is used to hold the large number of model parameters.  The `Scenario` class includes defaults  these can easily be changed and at runtime to experiments with different designs.

In [9]:
class Scenario:
    '''
    Container class for scenario parameters/arguments
    
    Passed to a model and its process classes
    '''
    def __init__(self, random_number_set=DEFAULT_RNG_SET):
        '''
        The init method sets up our defaults.
        
        Parameters:
        -----------
        random_number_set: int, optional (default=DEFAULT_RNG_SET)
            Set to control the initial seeds of each stream of pseudo
            random numbers used in the model.
        '''
        #sampling
        self.random_number_set = random_number_set
        self.init_sampling()
    
    def set_random_no_set(self, random_number_set):
        '''
        Controls the random sampling 
        Parameters:
        ----------
        random_number_set: int
            Used to control the set of psuedo random numbers
            used by the distributions in the simulation.
        '''
        self.random_number_set = random_number_set
        self.init_resourse_counts()
        self.init_sampling()

    def init_resourse_counts(self):
        '''
        Init the counts of resources to default values...
        '''
        self.n_triage = DEFAULT_N_TRIAGE
        self.n_reg = DEFAULT_N_REG
        self.n_exam = DEFAULT_N_EXAM
        self.n_trauma = DEFAULT_N_TRAUMA
        
        # non-trauma (1), trauma (2) treatment cubicles
        self.n_cubicles_1 = DEFAULT_N_CUBICLES_1
        self.n_cubicles_2 = DEFAULT_N_CUBICLES_2

    def init_sampling(self):
        '''
        Create the distributions used by the model and initialise 
        the random seeds of each.
        '''
        # create random number streams
        rng_streams = np.random.default_rng(self.random_number_set)
        self.seeds = rng_streams.integers(0, 999999999, size=N_STREAMS)

        # create distributions
        
        # Triage duration
        self.triage_dist = Exponential(DEFAULT_TRIAGE_MEAN, 
                                       random_seed=self.seeds[0])
        
        # Registration duration (non-trauma only)
        self.reg_dist = Lognormal(DEFAULT_REG_MEAN, 
                                  np.sqrt(DEFAULT_REG_VAR),
                                  random_seed=self.seeds[1])
        
        # Evaluation (non-trauma only)
        self.exam_dist = Normal(DEFAULT_EXAM_MEAN,
                                np.sqrt(DEFAULT_EXAM_VAR),
                                random_seed=self.seeds[2])
        
        # Trauma/stablisation duration (trauma only)
        self.trauma_dist = Exponential(DEFAULT_TRAUMA_MEAN, 
                                       random_seed=self.seeds[3])
        
        # Non-trauma treatment
        self.nt_treat_dist = Lognormal(DEFAULT_NON_TRAUMA_TREAT_MEAN, 
                                       np.sqrt(DEFAULT_NON_TRAUMA_TREAT_VAR),
                                       random_seed=self.seeds[4])
        
        # treatment of trauma patients
        self.treat_dist = Lognormal(DEFAULT_TRAUMA_TREAT_MEAN, 
                                    np.sqrt(DEFAULT_TRAUMA_TREAT_VAR),
                                    random_seed=self.seeds[5])
        
        # probability of non-trauma patient requiring treatment
        self.nt_p_treat_dist = Bernoulli(DEFAULT_NON_TRAUMA_TREAT_P, 
                                         random_seed=self.seeds[6])
        
        
        # probability of non-trauma versus trauma patient
        self.p_trauma_dist = Bernoulli(DEFAULT_PROB_TRAUMA, 
                                       random_seed=self.seeds[7])
        
        # arrivals
        self.arrival_dist = Exponential(DEFAULT_MEAN_IAT,
                                       random_seed=self.seeds[8])

## Patient Pathways Process Logic

`simpy` uses a process based worldview.  We can easily create whatever logic - simple or complex  for the model.  Here the process logic for trauma and non-trauma patients is seperated into two classes `TraumaPathway`  and `NonTraumaPathway`. 

In [10]:
class TraumaPathway:
    '''
    Encapsulates the process a patient with severe injuries or illness.
    
    These patients are signed into the ED and triaged as having severe injuries
    or illness.
    
    Patients are stabilised in resus (trauma) and then sent to Treatment.  
    Following treatment they are discharged.
    '''
    def __init__(self, identifier, env, args):
        '''
        Constructor method
        
        Params:
        -----
        identifier: int
            a numeric identifier for the patient.
            
        env: simpy.Environment
            the simulation environment
            
        args: Scenario
            Container class for the simulation parameters
            
        '''
        self.identifier = identifier
        self.env = env
        self.args = args
        
        # triage resource
        self.triage = args.triage
        
        # metrics
        self.arrival = -np.inf
        self.wait_triage = -np.inf
        self.wait_trauma = -np.inf
        self.wait_treat = -np.inf
        self.total_time = -np.inf
        
    def execute(self):
        '''
        simulates the major treatment process for a patient
        
        1. request and wait for sign-in/triage
        2. trauma
        3. treatment
        '''
        # record the time of arrival and entered the triage queue
        self.arrival = self.env.now

        # request sign-in/triage 
        with self.triage.request() as req:
            yield req
            
            # record the waiting time for triage
            self.wait_triage = self.env.now - self.arrival
            trace(f'patient {self.identifier} triaged to trauma '
                  f'{self.env.now:.3f}')
            
            # sample triage duration.
            triage_duration = args.triage_dist.sample()
            yield self.env.timeout(triage_duration)
            
            trace(f'triage {self.identifier} complete {self.env.now:.3f}; '
                  f'waiting time was {self.wait_triage:.3f}')
            
        # record the time that entered the trauma queue
        start_wait = self.env.now
    
        # request trauma room 
        with self.args.trauma.request() as req:
            yield req
            
            # record the waiting time for trauma
            self.wait_trauma = self.env.now - start_wait
            trace(f'stabilisation of patient {self.identifier} at '
                  f'{self.env.now:.3f}')
            
            # sample stablisation duration.
            trauma_duration = args.trauma_dist.sample()
            yield self.env.timeout(trauma_duration)
            
            trace(f'patient {self.identifier} stabilised {self.env.now:.3f}; '
                  f'waiting time was {self.wait_trauma:.3f}')
            
        # record the time that entered the treatment queue
        start_wait = self.env.now
    
        # request treatment cubicle 
        with self.args.cubicle_2.request() as req:
            yield req
            
            # record the waiting time for trauma
            self.wait_treat = self.env.now - start_wait
            trace(f'treatment of patient {self.identifier} at '
                  f'{self.env.now:.3f}')
            
            # sample treatment duration.
            treat_duration = args.trauma_dist.sample()
            yield self.env.timeout(treat_duration)
            
            trace(f'patient {self.identifier} treatment complete {self.env.now:.3f}; '
                  f'waiting time was {self.wait_treat:.3f}')
        
        # total time in system
        self.total_time = self.env.now - self.arrival     

In [11]:
class NonTraumaPathway(object):
    '''
    Encapsulates the process a patient with minor injuries and illness.
    
    These patients are signed into the ED and triaged as having minor 
    complaints and streamed to registration and then examination. 
    
    Post examination 40% are discharged while 60% proceed to treatment.  
    Following treatment they are discharged.
    '''
    def __init__(self, identifier, env, args):
        '''
        Constructor method
        
        Params:
        -----
        identifier: int
            a numeric identifier for the patient.
            
        env: simpy.Environment
            the simulation environment
            
        args: Scenario
            Container class for the simulation parameters
            
        '''
        self.identifier = identifier
        self.env = env
        self.args = args
        
        # triage resource
        self.triage = args.triage
        
        # metrics
        self.arrival = -np.inf
        self.wait_triage = -np.inf
        self.wait_reg = -np.inf
        self.wait_eval = -np.inf
        self.wait_treat = -np.inf
        self.total_time = -np.inf
        
    def execute(self):
        '''
        simulates the non-trauma/minor treatment process for a patient
        
        1. request and wait for sign-in/triage
        2. patient registration
        3. evaluation
        4.1 40% discharged
        4.2 60% treatment then discharge
        '''
        # record the time of arrival and entered the triage queue
        self.arrival = self.env.now

        # request sign-in/triage 
        with self.triage.request() as req:
            yield req
            
            # record the waiting time for triage
            self.wait_triage = self.env.now - self.arrival
            trace(f'patient {self.identifier} triaged to minors '
                  f'{self.env.now:.3f}')
            
            # sample triage duration.
            triage_duration = args.triage_dist.sample()
            yield self.env.timeout(triage_duration)
            
            trace(f'triage {self.identifier} complete {self.env.now:.3f}; '
                  f'waiting time was {self.wait_triage:.3f}')
            
        # record the time that entered the registration queue
        start_wait = self.env.now
    
        # request registration clert 
        with self.args.registration.request() as req:
            yield req
            
            # record the waiting time for registration
            self.wait_reg = self.env.now - start_wait
            trace(f'registration of patient {self.identifier} at '
                  f'{self.env.now:.3f}')
            
            # sample registration duration.
            reg_duration = args.trauma_dist.sample()
            yield self.env.timeout(reg_duration)
            
            trace(f'patient {self.identifier} registered at'
                  f'{self.env.now:.3f}; '
                  f'waiting time was {self.wait_reg:.3f}')
            
        # record the time that entered the evaluation queue
        start_wait = self.env.now
    
        # request evaluation resource
        with self.args.exam.request() as req:
            yield req
            
            # record the waiting time for registration
            self.wait_eval = self.env.now - start_wait
            trace(f'evaluation of patient {self.identifier} begins '
                  f'{self.env.now:.3f}')
            
            # sample evaluation duration.
            eval_duration = args.trauma_dist.sample()
            yield self.env.timeout(eval_duration)
            
            trace(f'patient {self.identifier} evaluation complete ' 
                  f'at {self.env.now:.3f};'
                  f'waiting time was {self.wait_eval:.3f}')
            
        # sample if patient requires treatment?
        self.require_treat = self.args.nt_p_treat_dist.sample()
        
        if self.require_treat:
        
            # record the time that entered the treatment queue
            start_wait = self.env.now
    
            # request treatment cubicle
            with self.args.cubicle_1.request() as req:
                yield req

                # record the waiting time for treatment
                self.wait_treat = self.env.now - start_wait
                trace(f'treatment of patient {self.identifier} begins '
                      f'{self.env.now:.3f}')

                # sample treatment duration.
                treat_duration = args.nt_treat_dist.sample()
                yield self.env.timeout(treat_duration)

                trace(f'patient {self.identifier} treatment complete '
                      f'at {self.env.now:.3f};'
                      f'waiting time was {self.wait_treat:.3f}')
                
        # total time in system
        self.total_time = self.env.now - self.arrival         

## Main model class

The main class that a user interacts with to run the model is `TreatmentCentreModel`.  This implements a `.run()` method, contains a process for patients arrivals and inits instances of `TraumaPathway` or `NonTraumaPathway` depending on the arrival type.

In [12]:
class TreatmentCentreModel:
    '''
    The treatment centre model
    
    Patients arrive at random to a treatment centre, are triaged
    and then processed in either a trauma or non-trauma pathway.
    '''
    def __init__(self, args):
        self.env = simpy.Environment()
        self.args = args
        self.init_resources()
        
        self.patients = []
        self.trauma_patients = []
        self.non_trauma_patients = []
        
        # performance metrics:
        self.wait_for_operator = 0.0
        self.wait_for_nurse = 0.0
        self.operator_util = 0.0
        self.nurse_util = 0.0
        self.operator_queue = 0.0
        self.nurse_queue = 0.0

        self.nurse_time_used = 0.0
        self.operator_time_used = 0.0

        self.n_calls = 0
        self.n_callbacks = 0

    def init_resources(self):
        '''
        Init the number of resources
        and store in the arguments container object
        
        Resource list:
        1. Sign-in/triage bays
        2. registration clerks
        3. examination bays
        4. trauma bays
        5. non-trauma cubicles (1)
        6. trauma cubicles (2)
         
        '''
        # sign/in triage
        self.args.triage = simpy.Resource(self.env, 
                                          capacity=self.args.n_triage)
        
        # registration
        self.args.registration = simpy.Resource(self.env, 
                                                capacity=self.args.n_reg)
        
        # examination
        self.args.exam = simpy.Resource(self.env, 
                                        capacity=self.args.n_exam)
        
        # trauma
        self.args.trauma = simpy.Resource(self.env, 
                                          capacity=self.args.n_trauma)
        
        # non-trauma treatment
        self.args.cubicle_1 = simpy.Resource(self.env, 
                                              capacity=self.args.n_cubicles_1)
        
        # trauma treatment
        self.args.cubicle_2 = simpy.Resource(self.env, 
                                              capacity=self.args.n_cubicles_2)
        
        
        
    def run(self, results_collection_period=DEFAULT_RESULTS_COLLECTION_PERIOD):
        '''
        Conduct a single run of the model in its current 
        configuration
        
        
        Parameters:
        ----------
        results_collection_period, float, optional
            default = DEFAULT_RESULTS_COLLECTION_PERIOD
            
        warm_up, float, optional (default=0)
        
            length of initial transient period to truncate
            from results.
            
        Returns:
        --------
            None
        '''
        # setup the arrival generator process
        self.env.process(self.arrivals_generator())
                
        # run
        self.env.run(until=results_collection_period)
        
    def arrivals_generator(self):
        '''
        Simulate the arrival of patients to the model
        
        Patients either follow a TraumaPathway or
        NonTraumaPathway simpy process.
        
        Implementation with as a stationary poisson process.
        '''
        for patient_count in itertools.count(start=1):
            
            # iat
            inter_arrival_time = self.args.arrival_dist.sample()
            yield self.env.timeout(inter_arrival_time)
            
            trace(f'patient {patient_count} arrives at: {self.env.now:.3f}')
            
            # sample if the patient is trauma or non-trauma
            trauma = self.args.p_trauma_dist.sample()
            if trauma:
                # create and store a trauma patient to update KPIs.
                new_patient = TraumaPathway(patient_count, self.env, self.args)
                self.trauma_patients.append(new_patient)
            else:
                # create and store a non-trauma patient to update KPIs.
                new_patient = NonTraumaPathway(patient_count, self.env, self.args)
                self.non_trauma_patients.append(new_patient)
                
            # start the pathway process for the patient
            self.env.process(new_patient.execute())

    def process_event(self, *args, **kwargs):
        '''
        Running calculates each time a Patient process ends
        (when a patient departs the simulation model)
        
        Params:
        --------
        *args: list
            variable number of arguments. This is useful in case you need to
            pass different information for different events
        
        *kwargs: dict
            keyword arguments.  Same as args, but you can is a dict so you can
            use keyword to identify arguments.
        
        '''
        patient = args[0]
        msg = args[1]
        
        #there are cleaner ways of implementing this, but 
        #for simplicity it is implemented as an if-then statement
        if msg == 'operator_service_complete':
            self.n_calls += 1
            n = self.n_calls
            
            #running calculation for mean operator waiting time
            self.wait_for_operator += \
                (patient.waiting_time - self.wait_for_operator) / n

            #running calc for mean operator utilisation
            self.operator_time_used += patient.call_duration

            #mean operator queue length
            current_q = len(self.args.operators.queue)
            self.operator_queue += (current_q - self.operator_queue) / n
            
        elif msg == 'nurse_service_complete':
            self.n_callbacks += 1
            n = self.n_callbacks
            
            #running calculation for mean nurse waiting time
            self.wait_for_nurse += \
                (patient.waiting_time_nurse - self.wait_for_nurse) / n

            #running calc for mean nurse utilisation
            self.nurse_time_used += patient.nurse_call_duration

            #mean nurse queue length
            current_q = len(self.args.nurses.queue)
            self.nurse_queue += (current_q - self.nurse_queue) / n
            
            
        #print(self.wait_for_operator)

    def run_summary_frame(self):
        #append to results df
        mean_waiting_time = self.wait_for_operator
        nurse_waiting_time = self.wait_for_nurse

        #adjust util calculations for warmup period
        rc_period = self.env.now - self.args.warm_up
        util = self.operator_time_used / (rc_period * self.args.n_operators)
        nurse_util = self.nurse_time_used / (rc_period * self.args.n_nurses)

        df = pd.DataFrame({'1':{'operator_wait': mean_waiting_time, 
                                'operator_queue': self.operator_queue,
                                'ops_util':util,
                                'nurse_wait':nurse_waiting_time,
                                'nurse_util':nurse_util,
                                'nurse_queue': self.nurse_queue}})
        df = df.T
        df.index.name = 'rep'
        return df

## Executing a model

We note that there are **many ways** to setup a `simpy` model and execute it (that is part of its fantastic flexibility).  The organisation of code we show below is based on our experience of using the package in practice. The approach also allows for easy parallisation over multiple CPU cores using `joblib`.

In [13]:
def single_run(scenario, rc_period=DEFAULT_RESULTS_COLLECTION_PERIOD, 
               random_no_set=DEFAULT_RNG_SET):
    '''
    Perform a single run of the model and return the results
    
    Parameters:
    -----------
    
    scenario: Scenario object
        The scenario/paramaters to run
        
    rc_period: int
        The length of the simulation run that collects results
        
    random_no_set: int or None, optional (default=DEFAULT_RNG_SET)
        Controls the set of random seeds used by the stochastic parts of the 
        model.  Set to different ints to get different results.  Set to None
        for a random set of seeds.
        
    Returns:
    --------
        pandas.DataFrame:
        results from single run.
    '''  
        
    # set random number set - this controls sampling for the run.
    scenario.set_random_no_set(random_no_set)

    # create an instance of the model
    model = TreatmentCentreModel(scenario)

    model.run(results_collection_period=rc_period)
    
    #run the model
    #results_summary = model.run_summary_frame()
    
    #return results_summary

In [14]:
def multiple_replications(scenario, rc_period=DEFAULT_RESULTS_COLLECTION_PERIOD, 
                          n_reps=5):
    '''
    Perform multiple replications of the model.
    
    Params:
    ------
    scenario: Scenario
        Parameters/arguments to configurethe model
    
    rc_period: float, optional (default=DEFAULT_RESULTS_COLLECTION_PERIOD)
        results collection period.  
        the number of minutes to run the model to collect results

    n_reps: int, optional (default=DEFAULT_N_REPS)
        Number of independent replications to run.
        
    Returns:
    --------
    pandas.DataFrame
    '''

    results = [single_run(scenario, rc_period, random_no_set=rep) 
               for rep in range(n_reps)]
    
    #format and return results in a dataframe
    df_results = pd.concat(results)
    df_results.index = np.arange(1, len(df_results)+1)
    df_results.index.name = 'rep'
    return df_results

### Single run of the model

In [18]:
# Change this to True to see a trace...
TRACE = False

# create the default scenario
args = Scenario()

# use the single_run() func
# try changing `random_no_set` to see different run results
print('Running simulation ...', end=' => ')
single_run(args, random_no_set=42)
print('simulation complete.')

Running simulation ... => simulation complete.


### Multiple independent replications